In [1]:
import requests
import pandas as pd
import random
import time
import datetime
from bs4 import BeautifulSoup
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

C:\Users\whitl\AppData\Local\Temp\ipykernel_10560\2788732232.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Retrieve API given client code
def get_auth_token():
    client_id = "c92bf1c7c40f4500a2b4e2a1faac4997"
    client_secret = "a00809b4c9734ea38fe0d4b05719b62c"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}"
    url = "https://accounts.spotify.com/api/token"

    response = requests.post(url=url, data=data, headers=headers)
    token = response.json()["access_token"]
    print(f"TOKEN {token} GENERATED")
    return {"Authorization": f"Bearer  {token}"}

auth = []
auth.append(get_auth_token())

TOKEN BQCEJirmmzSEtgIF-BQ_gChvin_4K22Lv9CtgT4R7VtBSAaT8PJton13h1gEOd_QueUbSk3fYrW0sbfdWNZiQixAUNtbEo_p0tJTEyEyLL5aHluCVEY GENERATED


In [3]:
def extract_artist(artist_json: dict):
    id = artist_json["id"]
    name = artist_json["name"]
    genres = artist_json["genres"]
    followers = artist_json["followers"]
    popularity = artist_json["popularity"]
    return {
        "id": id,
        "name": name,
        "genres": genres,
        "followers": followers,
        "popularity": popularity
        }

def get_artist(a_id: str, auth: list):
    artist_url = f"https://api.spotify.com/v1/artists/{a_id}"
    response = requests.get(url=artist_url, headers=auth[-1])

    # check for expired token
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_artist(a_id, auth)
    return response.json()

def get_related_artists(a_id: str, auth: list):
    related_url = f"https://api.spotify.com/v1/artists/{a_id}/related-artists"
    response = requests.get(url=related_url, headers=auth[-1])
    
    # check for expired token
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_related_artists(a_id, auth)

    return response.json()   

def extract_related_artists(related_json: dict, visited: set, sources: list):
    artists = []
    for a in related_json["artists"]:
        if a["id"] not in visited:
            visited.add(a["id"])
            sources.append(a["id"])
            artists.append(extract_artist(a))
    return artists

In [90]:
removal_list = []
def remove_artist(a_id):
    removal_list.append(a_id)    

In [91]:
def get_top_tracks(a_id: str, auth: list):
    url = f"https://api.spotify.com/v1/artists/{a_id}/top-tracks"
    response = requests.get(url, headers=auth[-1])
    
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_related_artists(a_id, auth[-1])

    # wait for clearance if request is denied for rate limiting reasons
    if response.status_code == 429:
        sleep = int(response.headers["retry-after"]) + 1
        if sleep > 1800:
            raise BaseException(f"API Limited for too long. Pausing. Try again in {sleep} seconds.")
        print(f"Rate limited. Sleeping for {sleep} seconds.")
        time.sleep(sleep)
        return get_top_tracks(a_id, auth)

    if response.status_code == 404:
        raise LookupError('Track not found')
    
    return response.json()

def extract_top_tracks(tracks_json):
    tracks = []
    for track in tracks_json["tracks"]:
        artists = track["artists"]
        duration = track["duration_ms"]
        explicit = track["explicit"]
        id = track["id"]
        name = track["name"]
        popularity = track["popularity"]
        row = {
            "id": id,
            "name": name,
            "artists": artists,
            "duration": duration,
            "explicit": explicit,
            "popularity": popularity
        }
        tracks.append(row)
    return tracks

In [31]:
# URL of the Wikipedia page
wiki_url = 'https://en.wikipedia.org/wiki/Coachella_Festival_line-ups'

# Fetch the web page content
response = requests.get(wiki_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table', class_='wikitable')

table_years = [1999, 1999, 2001, 2002, 2002, 2003, 2003, 2004, 2004, 2005, 2005, 2006, 2006, 2007, 2007, 2007, 2008, 2008, 2008, 2009, 2009, 2009, 2010, 2010, 2010, 2011, 2011, 2011, 2012, 2012, 2012, 2013, 2013, 2013, 2014, 2014, 2014, 2015, 2015, 2015, 2016, 2016, 2016, 2017, 2017, 2017, 2018, 2018, 2018, 2019, 2019, 2019, 2020, 2022, 2022, 2022, 2023, 2023, 2023]
performers_data = []

# Iterate through each table and extract its contents
for index, table in enumerate(tables, start=1):
    year = table_years[index - 1]
    if year == 2020:
        continue

    rows = table.find_all('tr')
    header_row = True
    headers = []
    performers = []
    for row in rows:
        cells = row.find_all(['th', 'td'])
        counter = 0
        for cell in cells:
            if header_row:
                headers.append(cell.get_text(strip=True))
            else:
                performers = cell.get_text(strip=False).split("\n")[1:-1]
                performers_data.append({
                    "stage": headers[counter],
                    "year": year,
                    "performers": performers
                    })
                counter += 1
        header_row = False

In [38]:
# function for name cleaning
def remove_parentheses(text):
    pattern = r'\([^)]*\)'
    clean_text = re.sub(pattern, '', text)
    return clean_text.strip()

In [46]:
# data transformation, creating intermediate tables
performers_df = pd.DataFrame(data=performers_data)
singular = performers_df.explode(column="performers")
singular["name"] = singular["performers"].apply(remove_parentheses)
singular = singular.drop(labels="performers", axis=1)

In [49]:
def get_artist_ids(artist_names : list):
    client_credentials_manager = SpotifyClientCredentials(client_id="c92bf1c7c40f4500a2b4e2a1faac4997",
                                                        client_secret='a00809b4c9734ea38fe0d4b05719b62c')
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    artist_dict = {}
    for artist_name in artist_names:
        results = sp.search(q='artist:' + artist_name, type='artist', limit=1)
        
        if results['artists']['items']:
            artist_id = results['artists']['items'][0]['id']
            artist_dict[artist_name] = artist_id
        else:
            artist_dict[artist_name] = None

    artist_df = pd.DataFrame([artist_dict]).T.reset_index()
    artist_df.columns = ['name', 'id']
    return artist_df 

In [50]:
performer_ids = get_artist_ids(singular["name"].to_list())

ValueError: You are trying to merge on object and int64 columns for key 'name'. If you wish to proceed you should use pd.concat

In [83]:
# combining the existing table with the performer_ids table
artist_dataset = pd.merge(performer_ids, singular, how="left", on="name")
artist_dataset = artist_dataset.drop_duplicates(subset=["name", "stage", "year"])
artist_dataset = artist_dataset.dropna(subset="id")
artist_dataset = artist_dataset.reset_index().drop(labels='index', axis=1)
artist_dataset

,name,id,stage,year
0,Beck,4obzFoKoKRHIphyHzJ35G3,Main Stage,1999
1,Beck,4obzFoKoKRHIphyHzJ35G3,Gobi Tent,2004
2,Beck,4obzFoKoKRHIphyHzJ35G3,Coachella Stage,2014
3,The Chemical Brothers,1GhPHrq36VKCY3ucVaZCfo,Main Stage,1999
4,The Chemical Brothers,1GhPHrq36VKCY3ucVaZCfo,Insomniac Tent,2001
...,...,...,...,...
2907,Cassian,1ChtRJ3f4rbv4vtz87i6CD,Yuma Tent,2023
2908,TSHA,2kLa7JZu4Ijdz1Gle2khZh,Yuma Tent,2023
2909,LP Giobbi,3oKnyRhYWzNsTiss5n4Z1J,Yuma Tent,2023
2910,Airrica,57sPl5iWgq5t6AscVbMTOW,Yuma Tent,2023


In [6]:
# save data to csv
date = datetime.datetime.today().strftime('%Y-%m-%d')
artist_dataset.to_csv(f"data/coachella_artists.csv")

NameError: name 'artist_dataset' is not defined

In [96]:
# get the top tracks for each artist in the dataset
unique_artists = set(artist_dataset["id"].to_list())
song_dataset = pd.DataFrame(columns=['id', 'name', 'artists', 'duration', 'explicit', 'popularity'])

out = "data/song_dataset_2.csv"

starting_id = "16QCJENzcdhwka9bTKYMVB"
starting = False
for a_id in unique_artists:
    if a_id == starting_id:
        starting = True
        continue
    
    if not starting:
        print("skipped")
        continue

    try:
        t = get_top_tracks(a_id, auth)
        new_songs = pd.DataFrame(extract_top_tracks(t))
        print(f"{len(new_songs)} songs ADDED for artist: {a_id}")
        song_dataset = pd.concat([song_dataset, new_songs], ignore_index=True)
    except LookupError as e:
        print(f"No songs found for artist: {a_id}, adding to list.")
        remove_artist(a_id)
    except BaseException as et:
        print(et)
        song_dataset.to_csv(out)
        break

song_dataset.to_csv(out)

skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped
skipped


In [23]:
coachella_perf = pd.read_csv("data/coachella_artists.csv")
coachella_artists = coachella_perf.drop_duplicates(subset='id')
id_list = coachella_artists['id'].to_list()
genres = pd.DataFrame(columns=['id', 'genres'])

In [18]:
coachella_artist_meta = []
for id in id_list:
    a_json = get_artist(id, auth)
    row = extract_artist(a_json)
    print(row)
    coachella_artist_meta.append(row)

{'id': '181bsRPaVXVlUKXrxwZfHK', 'name': 'Megan Thee Stallion', 'genres': ['houston rap', 'pop', 'r&b', 'rap', 'trap queen'], 'followers': {'href': None, 'total': 8958891}, 'popularity': 75}
{'id': '1AhjOkOLkbHUfcHDSErXQs', 'name': '88rising', 'genres': ['asian american hip hop'], 'followers': {'href': None, 'total': 972339}, 'popularity': 60}
{'id': '4fxd5Ee7UefO4CUXgwJ7IP', 'name': 'Giveon', 'genres': ['r&b'], 'followers': {'href': None, 'total': 3731383}, 'popularity': 73}
{'id': '4Uc8Dsxct0oMqx0P6i60ea', 'name': 'Conan Gray', 'genres': ['bedroom pop', 'pop', 'pov: indie'], 'followers': {'href': None, 'total': 9552638}, 'popularity': 76}
{'id': '3ycxRkcZ67ALN3GQJ57Vig', 'name': 'Masego', 'genres': ['alternative r&b', 'indie soul'], 'followers': {'href': None, 'total': 1360195}, 'popularity': 69}
{'id': '1gWjcmBsveEYMxOZ0VRi32', 'name': 'Koffee', 'genres': ['dancehall', 'dancehall queen', 'jamaican dancehall', 'modern reggae'], 'followers': {'href': None, 'total': 716570}, 'popularit

In [20]:
temp_data = pd.DataFrame(coachella_artist_meta)
temp_data.to_csv('data/coachella_artist_metadata.csv')

In [30]:
temp_data = temp_data.drop('name', axis=1)
final_coachella = coachella_perf.merge(temp_data, how='left', on='id')
final_coachella = final_coachella.drop('Unnamed: 0', axis=1)
final_coachella.to_csv('data/coachella_artists_full.csv')

In [36]:
empty_list_rows = final_coachella[final_coachella['genres'].apply(len) == 0]
not_found = empty_list_rows.drop_duplicates(subset='id')['id'].to_list()
not_found

['6Zt25gNJMRGE2huRS9vHde',
 '5Tug131UB4YlP9Qhe9jBif',
 '06me8YDBAnzsRJ7truezEx',
 '6AxANmwcOD8IEWBy3pNR32',
 '6I2891HPq8zEnBEuwc5iAP',
 '7MvrywVZhbvS6M5Zv9XUi3',
 '71hHL858askwXLzRWykTfV',
 '42SPXNOsQBxnOBIZwaneJH',
 '0dgik75LbWPxn1g9rfZIX2',
 '1e2BbR4jKwI4Pm7A3RbAFm',
 '453XMRg98NIje3nHmxFXsH',
 '08Cmz2Whs8UYTPT4Th0c8w',
 '0QJsUBH0PcoT2fYI1F1Oee',
 '6vUjpCAR58MB3Jq2TsKUmK',
 '06iccIeT6E02EBhf7NaDE6',
 '1DMEHmubKOvDWaLbyt2OBP',
 '4b0s5L5HkUI6jrFwEDuTur',
 '4ugwXaNvZVbB4jqhdIXzKj',
 '57SxomZx6rGWH3aJMAJxiu',
 '7AewIRoLzZqVPf3tbgO9Zq',
 '20a7iq8UVWB82arsmrJHFf',
 '4GwyYAQXedw9Rf6R5lPqiG',
 '26HhAGEjD8E9u9EFsVzbvy',
 '6n1t55WMsSIUFHrAL4mUsB',
 '6MRgVSEuvOMYh2BrQRrZKX',
 '1Wkm5Jz5ZmVSE4idjPOfJp',
 '5aD0uh1BpYTN7AzhiSDgaO',
 '2Jk7qucsiK8zBMDpwpIdos',
 '0tJWNfrj8YdGsouHyBYlUa',
 '1gV480zBnPy4ktKHfHKxZ5',
 '4iuyDOnAmE11mVV4aRkLoI',
 '1FbrKRver8Lw1Wo9HHE6ag',
 '2XiSjEF8tz3hBWGSkqgQxs',
 '0HVw0wS9ujZ77XC1IkurQ9',
 '7iAUZj65TsyjfcvprSYb9w',
 '6GQPBUh0Ui1nfs0qf54wZ1',
 '1UQ5GQDdYPKgbIEn9sMiSg',
 